In [69]:
import pandas as pd
from tqdm import tqdm
import json
import requests
from pathlib import Path
import re
import random
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from send2trash import send2trash
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [2]:
def download_pdf(doi, dest_folder, content):
    doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
    doi_suffix = re.sub(r"""([()/\\*,"': ?;<>])""", '_._', doi_suffix)
    filename = dest_folder + '/' + str(doi_suffix)+'.pdf'
    with open(filename, mode = 'wb') as f:
        f.write(content)

In [9]:
def get_publisher_download_url(crossref_resp, intended_app='text-mining'):
    # search for 'text-mining' in the intended-application field
    res = []
    for link in crossref_response.json()['link']:
        if re.search(intended_app,link['intended-application']):
            res.append(re.search(intended_app, link['intended-application']))
        else:
            res.append(None)
    # find the index of match obects in crossref link information
    publisher_api_idxs = np.where(np.array(res) != None)[0].tolist()
    # publisher_api_idxs
    links = [crossref_response.json()['link'][i] for i in publisher_api_idxs]
    # for now use the first link if there are more than one link
    full_text_link = links[0]['URL']
    return full_text_link

In [3]:
pdf_file_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*/*.pdf'))
pdfs = [path.stem for path in pdf_file_paths]

In [72]:
articles.to_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/sage_non_oa.csv',
                index=False)

In [5]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
articles['doi_suffix'] = articles['doi'].apply(lambda x: re_doi_suffix.search(x).group())
full_texts_to_download = [x for x in articles['doi_suffix'].tolist() if x not in pdfs]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), articles, how='inner', on='doi_suffix')
merge.shape

(230, 50)

In [6]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'
accept = 'application/vnd.citationstyles.csl+json, application/vnd.crossref.unixref+xml'
dest_folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/sage_non_oa'

In [66]:
n = random.randint(0, merge.shape[0])
row = merge.loc[n,:]
doi = row['doi']
doi

'10.3113/fai.2007.0715'

In [67]:
doi = row['doi']
out = {'doi': doi}
doi_url = 'https://doi.org/' + str(doi)
crossref_headers = {'User-Agent': user_agent, 'Accept': accept}
crossref_response = requests.get(
                url = doi_url,
                headers=crossref_headers,
                allow_redirects=True,
                verify=True)
out.update({'CR_status_code': crossref_response.status_code})
if crossref_response.status_code == 200:
    full_text_url = get_publisher_download_url(crossref_response, intended_app='similarity-checking')
    

In [65]:
# print(driver.current_url)
full_text_resp = requests.get(url = driver.current_url,
                              headers = {'User-Agent': user_agent})
full_text_resp.status_code

503

In [56]:
full_text_resp = requests.get(url = full_text_url, headers = {'User-Agent': user_agent}, allow_redirects=True)
full_text_resp.status_code

503

In [ ]:
for idx, row in tqdm(merge.iterrows(), total=merge.shape[0]):
    doi = row['doi']
    out = {'doi': doi}
    doi_url = 'https://doi.org/' + str(doi)
    crossref_headers = {'User-Agent': user_agent, 'Accept': accept}
    try:
        crossref_response = requests.get(
                        url = doi_url,
                        headers=crossref_headers,
                        allow_redirects=True,
                        verify=True)
        out.update({'CR_status_code': crossref_response.status_code})
        if crossref_response.status_code == 200:
            full_text_url = get_publisher_download_url(crossref_response, intended_app='similarity-checking')
            driver.get(full_text_url)
            time.sleep(3)
            full_text_resp = requests.get(url = driver.current_url,
                                          headers = {'User-Agent': user_agent},
                                         allow_redirects=True)
            out.update({'full_text_SC': full_text_resp.status_code})
            if full_text_resp.status_code == 200:
                download_pdf(doi=doi, dest_folder=dest_folder, content=full_text_resp.content)
    except Exception as e:
        print(e)
        out.update({'error': e})
    log.append(out)